In [1]:
import os,re
from collections import defaultdict
from igf_data.illumina.samplesheet import SampleSheet
import pandas as pd
from igf_data.igfdb.igfTables import Base, Experiment, Run
from sqlalchemy import create_engine
from igf_data.igfdb.baseadaptor import BaseAdaptor
from igf_data.igfdb.platformadaptor import PlatformAdaptor
from igf_data.igfdb.projectadaptor import ProjectAdaptor
from igf_data.igfdb.seqrunadaptor import SeqrunAdaptor
from igf_data.igfdb.sampleadaptor import SampleAdaptor
from igf_data.igfdb.experimentadaptor import ExperimentAdaptor
from igf_data.igfdb.runadaptor import RunAdaptor
from igf_data.igfdb.collectionadaptor import CollectionAdaptor
from igf_data.igfdb.fileadaptor import FileAdaptor
from igf_data.utils.fileutils import calculate_file_checksum

In [2]:
dbparams={'dbname':'../../test_dir/test9_collect_fastq/test.db'}

In [3]:
base=BaseAdaptor(**dbparams)
Base.metadata.drop_all(base.engine)
Base.metadata.create_all(base.engine)
session_class=base.get_session_class()

In [4]:
platform_data=[{'platform_igf_id':'ILM4K_001', \
                'model_name':'HISEQ4000',\
                'vendor_name':'ILLUMINA', \
                'software_name':'RTA', \
                'software_version':'RTA2'},
               {'platform_igf_id':'NB501820', \
                'model_name':'NEXTSEQ',\
                'vendor_name':'ILLUMINA', \
                'software_name':'RTA', \
                'software_version':'RTA2'},
               {'platform_igf_id':'ILMMS_001', \
                'model_name':'MISEQ',\
                'vendor_name':'ILLUMINA', \
                'software_name':'RTA', \
                'software_version':'RTA1.18.64'},
              ]

In [5]:
pl=PlatformAdaptor(**{'session_class':base.session_class})
pl.start_session()
pl.store_platform_data(data=platform_data)

In [6]:
project_data=[{'project_igf_id':'ferrer_t2dnoncod-hybcap', \
       'project_name':'ferrer_t2dnoncod-hybcap',  \
       'description':'Its project C', \
       'project_deadline':'Before August 2017', \
       'comments':'Some samples are treated with drug X'}
     ]

In [7]:
pa=ProjectAdaptor(**{'session_class':base.session_class})
pa.start_session()
pa.store_project_and_attribute_data(project_data)

In [8]:
sample_data=[{'sample_igf_id':'IGF0007142_QXT', \
              'taxon_id':'9606',\
              'scientific_name':'Homo sapiens',\
              'common_name':'human',\
              'donor_anonymized_id':'donor_001',\
              'description':'Sample A from donor 001',\
              'phenotype':'Healthy',\
              'sex':'FEMALE',\
              'project_igf_id':'ferrer_t2dnoncod-hybcap',\
              'sample_tube':'tube001',\
              'sample_library':'IGFS0001_20170628'},
             {'sample_igf_id':'IGF0007143_QXT', \
              'taxon_id':'9606',\
              'scientific_name':'Homo sapiens',\
              'common_name':'human',\
              'donor_anonymized_id':'donor_002',\
              'description':'Sample B from donor 002',\
              'phenotype':'Cancer',\
              'sex':'FEMALE',\
              'project_igf_id':'ferrer_t2dnoncod-hybcap',\
              'sample_tube':'tube002',\
              'sample_library':'IGFS0002_20170628'},
             {'sample_igf_id':'IGF0007144_QXT', \
              'taxon_id':'9606',\
              'scientific_name':'Homo sapiens',\
              'common_name':'human',\
              'donor_anonymized_id':'donor_002',\
              'description':'Sample B from donor 002',\
              'phenotype':'Cancer',\
              'sex':'FEMALE',\
              'project_igf_id':'ferrer_t2dnoncod-hybcap',\
              'sample_tube':'tube002',\
              'sample_library':'IGFS0002_20170628'},
            ]

In [9]:
sa=SampleAdaptor(**{'session_class':base.session_class})
sa.start_session()
sa.store_sample_and_attribute_data(data=sample_data)

In [10]:
seqrun_data=[{'seqrun_igf_id':'171006_NB501820_0009_AHTGYKAFXX', 
              'flowcell_id':'HTGYKAFXX', 
              'platform_igf_id':'NB501820',},
            ]

In [11]:
sra=SeqrunAdaptor(**{'session_class':base.session_class})
sra.start_session()
sra.store_seqrun_and_attribute_data(data=seqrun_data)

## runnable

In [12]:
samplesheet_filename='SampleSheet.csv'
seqrun_igf_id='171006_NB501820_0009_AHTGYKAFXX'
model_name='NEXTSEQ'
flowcell_id='HTGYKAFXX'
file_location='HPC_PROJECT'
fastq_dir='../../test_dir/test9_collect_fastq/nextseq_test/fastq/1_16'

In [13]:
def get_fastq_and_samplesheet(fastq_dir, samplesheet_filename):
    r1_fastq_regex=re.compile(r'\S+_R1_\d+\.fastq(\.gz)?', re.IGNORECASE)
    r2_fastq_regex=re.compile(r'\S+_R2_\d+\.fastq(\.gz)?', re.IGNORECASE)
    
    samplesheet_list=list()
    r1_fastq_list=list()
    r2_fastq_list=list()
    
    for root, dirs, files in os.walk(top=fastq_dir, topdown=True):
        if samplesheet_filename in files:
            samplesheet_list.append(os.path.join(root,samplesheet_filename))
        for file in files:
            if r1_fastq_regex.match(file):
                r1_fastq_list.append(os.path.join(root,file))
            elif r2_fastq_regex.match(file):
                r2_fastq_list.append(os.path.join(root,file))
                
    if len(r2_fastq_list) > 0 and len(r1_fastq_list) != len(r2_fastq_list):
        raise ValueError('R1 {0} and R2 {1}'.format(len(r1_fastq_list),len(r2_fastq_list)))
        
    if len(samplesheet_list) > 1:
        raise ValueError('Found more than one samplesheet file for fastq dir {0}'.format(fastq_dir))
        
    return samplesheet_list[0], r1_fastq_list, r2_fastq_list

In [14]:
def link_fastq_file_to_sample(sample_name,r1_fastq_list, r2_fastq_list):
    sample_files=defaultdict(lambda: defaultdict(lambda: defaultdict()))
    r1_regex=re.compile(sample_name+'_S\d+_L(\d+)_R1_\d+\.fastq(\.gz)?',re.IGNORECASE)
    for file1 in r1_fastq_list:
        if r1_regex.match(os.path.basename(file1)):
            m=r1_regex.match(os.path.basename(file1))
            lane_id=m.group(1).strip('0')
            sample_files[lane_id]['R1']=file1
            
    if len(r2_fastq_list) > 0:
        r2_regex=re.compile(sample_name+'_S\d+_L(\d+)_R2_\d+\.fastq(\.gz)?',re.IGNORECASE)
        for file2 in r2_fastq_list:
            if r2_regex.match(os.path.basename(file2)):
                m=r2_regex.match(os.path.basename(file2))
                lane_id=m.group(1).strip('0')
                sample_files[lane_id]['R2']=file2
    return sample_files

In [15]:
def collect_fastq_and_sample_info(fastq_dir,samplesheet_filename,seqrun_igf_id,model_name):
    (samplesheet_file, r1_fastq_list, r2_fastq_list)=get_fastq_and_samplesheet(fastq_dir, samplesheet_filename)
    samplesheet_data=SampleSheet(infile=samplesheet_file)
    fastq_files_list=list()
    for row in samplesheet_data._data:
        sample_name=row['Sample_Name']
        sample_id=row['Sample_ID']
        project_name=row['Sample_Project']
        description=row['Description']
        sample_files=link_fastq_file_to_sample(sample_name,r1_fastq_list, r2_fastq_list)
        for lane, lane_files in sample_files.items():
            fastq_info={'sample_igf_id':sample_id,
                        'sample_name':sample_name,
                        'project_igf_id':project_name,
                        'lane_number':lane,
                        'seqrun_igf_id':seqrun_igf_id,
                        'platform_name':model_name,
                        'flowcell_id':flowcell_id,
                        'description':description
                        }
            for read_type, filepath in lane_files.items():
                fastq_info.update({read_type:filepath})     # allowing only one file per lane per read type
            fastq_files_list.append(fastq_info)             # adding entries per samle per lane
    return fastq_files_list

In [16]:
fastq_files_list=collect_fastq_and_sample_info(fastq_dir,\
                                               samplesheet_filename,\
                                               seqrun_igf_id,\
                                               model_name)

In [17]:
pd.DataFrame(fastq_files_list).head(2)

,R1,R2,description,flowcell_id,lane_number,platform_name,project_igf_id,sample_igf_id,sample_name,seqrun_igf_id
0,../../test_dir/test9_collect_fastq/nextseq_tes...,../../test_dir/test9_collect_fastq/nextseq_tes...,,HTGYKAFXX,4,NEXTSEQ,ferrer_t2dnoncod-hybcap,IGF0007142_QXT,ctrl_91_H7_MP3913_QXT,171006_NB501820_0009_AHTGYKAFXX
1,../../test_dir/test9_collect_fastq/nextseq_tes...,../../test_dir/test9_collect_fastq/nextseq_tes...,,HTGYKAFXX,2,NEXTSEQ,ferrer_t2dnoncod-hybcap,IGF0007142_QXT,ctrl_91_H7_MP3913_QXT,171006_NB501820_0009_AHTGYKAFXX


## experiment utils

In [18]:
def calculate_experiment_run_and_file_info(data,restricted_list):
    if not isinstance(data, pd.Series):
        data=pd.Series(data)
        
    # set library id
    library_id=None
    if data.description and data.description not in restricted_list:
        library_id=data.description                                    
    else:
        library_id=data.sample_igf_id                                      
    
    # calcaulate experiment id
    experiment_id='{0}_{1}'.format(library_id,data.platform_name)         
    data['library_name']=library_id
    data['experiment_igf_id']=experiment_id
    # calculate run id
    run_igf_id='{0}_{1}_{2}'.format(experiment_id, \
                                    data.flowcell_id, \
                                    data.lane_number)
    data['run_igf_id']=run_igf_id
    # set collection name and type
    data['name']=run_igf_id
    data['type']='demultiplexed_fastq'
    data['location']='HPC_PROJECT'
    # set file md5 and size
    if 'R1' in data:
      data['R1_md5']=calculate_file_checksum(filepath=data.R1, \
                                             hasher='md5')
      data['R1_size']=os.path.getsize(data.R1)
    if 'R2' in data:
      data['R2_md5']=calculate_file_checksum(filepath=data.R2, \
                                             hasher='md5')
      data['R2_size']=os.path.getsize(data.R2)
    # set library strategy
    library_layout='SINGLE'
    if 'R1' in data and 'R2' in data and \
    data.R1 is not None and data.R2 is not None:
        library_layout='PAIRED'
    data['library_layout']=library_layout
    return data

In [19]:
def reformat_file_group_data(data):
    if isinstance(data, pd.DataFrame):
        data=data.to_dict(orient='records')
        
    if not isinstance(data,list):
        raise ValueError('Expecting list got {0}'.format(type(data)))
        
    reformatted_file_group_data=list()
    reformatted_file_data=list()

    for row in data:
      collection_name=None
      collection_type=None
      file_location=None
      if 'name' in row.keys():
        collection_name=row['name']
      if 'type' in row.keys():
        collection_type=row['type']
      if 'location' in row.keys():
        file_location=row['location']
      if 'R1' in row.keys():
        r1_file_path=row['R1']
        r1_file_size=row['R1_size'] if 'R1_size' in row.keys() else None
        r1_file_md5=row['R1_md5'] if 'R1_md5' in row.keys() else None
        reformatted_file_data.append({'file_path':r1_file_path,
                                      'md5':r1_file_md5,
                                      'location':file_location,
                                      'size':r1_file_size})
        reformatted_file_group_data.append({'name':collection_name,
                                            'type':collection_type,
                                            'file_path':r1_file_path})
      if 'R2' in row.keys():
        r2_file_path=row['R2']
        r2_file_size=row['R2_size'] if 'R2_size' in row.keys() else None
        r2_file_md5=row['R2_md5'] if 'R2_md5' in row.keys() else None
        reformatted_file_data.append({'file_path':r2_file_path,
                                      'md5':r2_file_md5,
                                      'location':file_location,
                                      'size':r2_file_size})
        reformatted_file_group_data.append({'name':collection_name,
                                            'type':collection_type,
                                            'file_path':r2_file_path})
    return pd.DataFrame(reformatted_file_data), pd.DataFrame(reformatted_file_group_data)

In [20]:
def build_and_store_exp_run_and_collection_in_db(session_class,fastq_files_list, restricted_list=['10X']):
    dataframe=pd.DataFrame(fastq_files_list)
    # calculate additional detail
    dataframe=dataframe.apply(lambda data: \
                              calculate_experiment_run_and_file_info(data, 
                                                                     restricted_list),\
                              axis=1)
    # get file data
    file_group_columns=['name','type','location',
                        'R1','R1_md5','R1_size','R2',
                        'R2_md5','R2_size']
    file_group_data=dataframe.loc[:,file_group_columns]
    file_group_data=file_group_data.drop_duplicates()
    (file_data,file_group_data)=reformat_file_group_data(data=file_group_data)
    
    # get base session
    base=BaseAdaptor(**{'session_class':session_class})
    base.start_session()
    # get experiment data
    experiment_columns=base.get_table_columns(table_name=Experiment, \
                                            excluded_columns=['experiment_id', 
                                                              'project_id', 
                                                              'sample_id' ])
    experiment_columns.extend(['project_igf_id', 
                               'sample_igf_id'])
    exp_data=dataframe.loc[:,experiment_columns]
    exp_data=exp_data.drop_duplicates()
    # get run data
    run_columns=base.get_table_columns(table_name=Run, \
                                       excluded_columns=['run_id', 
                                                         'seqrun_id', 
                                                         'experiment_id',
                                                         'date_created',
                                                         'status'
                                                        ])
    run_columns.extend(['seqrun_igf_id', 
                        'experiment_igf_id'])
    run_data=dataframe.loc[:,run_columns]
    run_data=run_data.drop_duplicates()
    
    # get collection data
    collection_columns=['name',
                        'type']
    collection_data=dataframe.loc[:,collection_columns]
    collection_data=collection_data.drop_duplicates()
    
    try:
      # store experiment to db
      ea=ExperimentAdaptor(**{'session':base.session})
      ea.store_project_and_attribute_data(data=exp_data,autosave=False)
      base.session.flush()
      # store run to db
      ra=RunAdaptor(**{'session':base.session})
      ra.store_run_and_attribute_data(data=run_data,autosave=False)
      base.session.flush()
      # store file to db
      fa=FileAdaptor(**{'session':base.session})
      fa.store_file_and_attribute_data(data=file_data,autosave=False)
      base.session.flush()
      # store collection to db
      ca=CollectionAdaptor(**{'session':base.session})
      ca.store_collection_and_attribute_data(data=collection_data,\
                                             autosave=False)
      base.session.flush()
      ca.create_collection_group(data=file_group_data,autosave=False)
      base.commit_session()
    except:
        base.rollback_session()
        raise
    finally:
      base.close_session()

In [21]:
build_and_store_exp_run_and_collection_in_db(session_class,fastq_files_list)